## AKI Calculation - Description

This notebook is our first attempt to identify the subjects with AKI in our cohort, based on the KDIGO SCr criteria.

The code to calculate AKI within the VPS-PN cohort can be found here: https://github.com/Lab-for-Integrated-Decision-Support/vps-peds-mods. The relevant code and folders are:

- r-markdown/AKI_over_3_day_LOS.Rmd - Processing markdown for AKI over the first 72 hrs (3 days) of ICU admission
- scripts/f_aki_functions.R - R file containing function definitions

**Note** This was copied over from Synapse Analytics workspace AKI Calc on 2024-01-10. None of the Spark functions were converted to Python yet.

The general process is as follows:

- Load the necessary data files, including ICU admissions, hospital admissions, laboratory results, and vial signs (mostly in `join_data(...)` function in the R code)
- Select, clean, and join the data files as needed
  - Remove those with a LOS < 12 hours, as we are calculating risk at 12 hrs
  - Ensure the weight and height are within the proper distribution
  - Ensure the creatinines are within the proper distribution
  - Filter for age < 18 
- Compute the baseline SCr (look in the aki_compute_bscr(...) function in the R code)
  - Requires functions for height-dependent and height-independent from Hessey 2017 article
  - Also requires Schwartz min/max estimates for ensuring we are within an appropriate range
  - Use logic for computing based on lines ~ 360 - 375 in f_aki_functions.R script

- From that, compute the AKI stage for each SCr that resulted between 0 and 72 hrs of hospital admission
  - Use KDIGO SCr criteria, found on lines 430 - 435 of f_aki_functions.R file
  - If AKI > stage 0 (no AKI) before 12 hrs, then patient is excluded for having AKI on admisison
  - Otherwise, look for highest stage achieved between 12 and 72 hrs - this is our stage label for that patient
- Summarize AKI presence in cohort and mortality among those with AKI at 72 hrs
- Gather covariates - recall that for the original model, this includes "cardiac arrest pre-admission" which we will have to get from diagnoses codes, and "post-operative" which we likely cannot include.
- Calculate AKI risk based on the Sanchez-Pinto 2016 coefficients on the covariates
  - Generate an ROC curve based on this score and report the AUROC
  - Calculate the thresholds at 50% and 90% and report results

### Initialize and Load Functions

First we set the folder path, and define the function `load_data_file` to load the specific PDC data file from this folder.

In [1]:
import pandas as pd
import numpy
import os, sys, importlib 

try:
    import pyspark
    import pyspark.pandas as ps
except:
    %pip install pyspark
    import pyspark
    import pyspark.pandas as ps

try:
    import plotly.express as px
except:
    %pip install plotly
    import plotly.express as px

# try:
#     import spark
# except:
#     %pip install spark
#     import spark

# import data_utils (customized library that write or delete files in blob storage) 
# import data_utils
import azure_BlobStorageConnection


#importlib.reload(data_utils) # Not needed unless we are modifying this code in this session

# import my load utilities
import load_utils

#importlib.reload(load_utils) # Not needed unless we modify this code in this session

Set environment variables using script.

In [2]:
from set_env_vars import set_all_env_vars

set_all_env_vars()

Add global `debug_` variables to determine when to print out counts, show data tables, and plot graphs. This speeds processing in the PySpark environment, but may not matter if we're working solely with pandas data frames.

In [3]:
debug_count = True
debug_show = True
debug_plot = True

Initialize spark session.

In [4]:
spark = pyspark.sql.SparkSession.builder.appName('creatinine').getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


24/09/23 17:45:56 WARN Utils: Your hostname, sdrury-compute resolves to a loopback address: 127.0.0.1; using 10.0.0.4 instead (on interface eth0)
24/09/23 17:45:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/23 17:45:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load pandas data frames of the four necessary data files.

In [5]:
import azureml.fsspec

uri = 'azureml://subscriptions/b06a3f28-16e7-4bcd-ab95-f192daca58db/resourcegroups/pdc-rg-westus-research/workspaces/pdc-mlw-westus-research1/datastores/pdc_ci_westus_research_readwrite_iam_link/paths/meds-wg/'

labs_lim = pd.read_csv(
    uri + 'labs.csv', 
    usecols=["pdc_hid", "pdc_pid", "resulttime", "pdc_name", "source_units", "source_value"]
)

Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


/tmp/ipykernel_5360/409669761.py:5: DtypeWarning: Columns (6,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  labs_lim = pd.read_csv(


In [7]:
labs_lim['pdc_name'].unique()

array(['ALBUMIN', 'ALP', 'ALT', 'AST', 'BUN', 'CALCIUM_TOT',
       'CALCIUM_ION', 'CHLORIDE', 'FIBRINOGEN', 'GLUCOSE', 'MAGNESIUM',
       nan, 'MCV', 'POTASSIUM', 'PT', 'PTT', 'SODIUM', 'WBC',
       'HEMOGLOBIN', 'PLTS', 'RDW', 'LYMPHO_PCT', 'NEUTRO_PCT',
       'BANDS_PCT', 'BILIRUBIN_TOT', 'BICARBONATE', 'PROTEIN', 'INR',
       'PH_ART', 'PCO2_ART', 'PO2_ART', 'BASE_EXC', 'PH_VEN', 'PCO2_VEN',
       'PO2_VEN', 'CREATININE', 'LACTATE', 'CRP', 'PH_CAP', 'PCO2_CAP',
       'PO2_CAP', 'CULT_URINE', 'GGT', 'PHOSPHORUS', 'CULT_OTHER',
       'CULT_BLOOD', 'INFLUENZA_A', 'INFLUENZA_B', 'PARAINFLU',
       'RHINO_ENTERO', 'HMPV', 'ADENOVIRUS', 'CULT_RESP', 'LDH',
       'BILIRUBIN_DIR', 'FERRITIN', 'ESR', 'LIPASE', 'AMYLASE',
       'CULT_CSF', 'RESP_VIRAL_PCR', 'ANC', 'ALC', 'HGB', 'D_DIMER',
       'PROCALCITONIN', 'PCO2', 'BANDS_ABS', 'CORONAVIRUS', 'RSV',
       'CORTISOL', 'SARS_COV_2', 'INFLUENZA', 'CULTURE', 'BASE_DEF'],
      dtype=object)

### Load `base_cohort` and `height_join_lim_mod` dataframe from previous notebook

`base_cohort` is a dataframe created in notebook `aki_calc_base_cohort.ipynb` that describes the patients in our cohort. We will use this dataframe to select only relevant rows from larger databases to be used in later calculations.

`height_join_lim_mod` is a dataframe containing information about the heights of patients in our cohort.

Note that `base_cohort.csv` should be in the same directory that holds the `vps-peds-aki` repo.

In [1]:
base_cohort = pd.read_csv(os.environ['AKI_DATA_PDC'] + 'base_cohort_v2.csv')
height_join_lim_mod = spark.read.parquet('../../height_join_lim_mod.parquet')

sites = base_cohort['site_id'].unique()

### Identify Creatinine Values

Next we pull out the creatinine values from the `labs_lim` data frame, restrict to `pdc_hid` values in the base cohort, and limit to those which resulted between the ICU Start and ICU End times. Because each row in `base_cohort` does not necessarily have a unique `pcd_hid` (a patient may be admitted to the ICU several times during a hospitalization), we have to left join with the base_cohort `pcd_hid` and then limit by the times to find the `pcd_eid` curresponding to that SCr lab value result.

**Note:** We are NOT limiting by ICU start and end time, because we will need SCr values from prior to the ICU start time to consider for the baseline SCr values.

**Also note:** There are many values in the `source_value` column that do not correspond to a single number (e.g. 'N/A', '< 0.02'). For now these will be dropped, but later a method for obtaining numeric values for some of these may be implemented.

In [ ]:
# Filter the labs to the SCr 
labs_scr = labs_lim[labs_lim['pdc_name']=='CREATININE']

def get_float_if_numeric_str(str_in):
    try:
        return float(str_in)
    except:
        return numpy.nan
    
labs_scr['val_numeric'] = [get_float_if_numeric_str(value) for value in labs_scr['source_value']]
labs_scr = labs_scr[~labs_scr['val_numeric'].isna()]
labs_scr['n_res_time'] = labs_scr['resulttime'].astype('long')
labs_scr = labs_scr.drop_duplicates()

if debug_count: print('\nNumber of rows of all Scr values:',labs_scr.count())

# Join to base cohort
scr_join_left = (
    base_cohort \
    .merge(labs_scr, on=['pdc_hid', 'pdc_pid'], how='left') \
    .drop_duplicates())
if debug_count: print('\nNumber of rows in left join:',scr_join_left.count())

# Filter to include only those with the time in between the ICU admission and discharge time
scr_join_left_icu = scr_join_left[
    (scr_join_left['n_res_time'] > scr_join_left['n_icuadmit']) \
    & (scr_join_left['n_res_time'] <= scr_join_left['n_icudc'])
]
if debug_count: print(\n'Number of filtered rows in ICU-only dataset:',scr_join_left_icu.count())

if debug_show: scr_join_left.head()

Now we make a box plot of the ICU SCr values by site, to ensure they appear similar (we cannot plot the full SCr dataset because the memory requirements are too great for the KL2Small machine).

In [ ]:
if debug_plot:

    fig = px.box(scr_join_left_icu[["site_id", "val_numeric"]], x="site_id", y="val_numeric", 
        labels={
            "siteid": "Site ID",
            "val_numeric": "Serum Creatinine Value"
        },
        log_y=True)
    fig.update_xaxes(type='category')
    fig.show()

### Baseline Creatinine

Now that we have SCr and height (for a subset of rows in our `base_cohort`), we have to calculate the baseline creatinine. This is done based on the following order:

- Prior creatinine within 6 months (up to 1 cay - can't use 12 hrs due to granularity) of ICU admission, as long as it falls above the lower limit of the Schwartz values (Mean - 2*SD)
- Height-dependent formula, if a height is given
- Height-independent formula, if no height is given

Recall that our data frame for creatinine values, `scr_join_left`, has all SCr values along with an associated result time (seconds from admission), age at admission, and patient ID. We need to collect all creatinines associated with each PID and the age at result (days will be the most granular we can get), then for each tuple (`hid`, `eid`) we look back 6 months through 1 day prior and attempt to identify the most recent baseline creatinine. The result may be None if there are no SCr values meeting that date restrictor for that tuple.

In [ ]:
# First we generate the table by PID which contains the creatinine values and 
# age (fraction of days)

scr_dates = scr_join_left[[
    "pdc_pid", "ageatadmission", "val_numeric", "n_res_time"
]]

scr_dates = scr_dates[
    (~scr_dates['val_numeric'].isna()) & \
    (~scr_dates['ageatadmission'].isna())
]

scr_dates['n_age_days'] = scr_dates['ageatadmission'].astype('long')
scr_dates['age_res_time'] = scr_dates['n_age_days'] + (scr_dates['n_res_time'] * (60. * 60. * 24.))

scr_dates = scr_dates[[
    'pdc_pid',
    'age_res_time',
    'val_numeric'
]]

if debug_count: print('Number of rows of SCr values:',len(scr_dates))
if debug_show: scr_dates.head()

# Now we join this to the base cohort by PID and compute # (fraction) of days difference
# between ICU admission and each SCr value
old_scr_vals = base_cohort.merge(
    scr_dates,
    on='pdc_pid',
    how='left'
)

old_scr_vals['n_age_days'] = old_scr_vals['ageatadmission'].astype('long')
old_scr_vals['icu_admit_days'] = old_scr_vals['n_age_days'] + (old_scr_vals['n_icuadmit'] * (60. * 60. * 24.))
old_scr_vals['res_before_admit'] = old_scr_vals['icu_admit_days'] - old_scr_vals['age_res_time']

old_scr_vals = old_scr_vals[old_scr_vals['res_before_admit'] >= 0]

In [ ]:
old_scr_vals.head()

Above we generated a data frame which has multiple rows per unique base_cohort tuple (`hid`, `eid`), and each row has an SCr value and a `res_before_admit` value, which is the number of days before the ICU admisison that the result was reported.

Here we filter by 0.5 days and 6 months, order by `res_before_admit` (ascending), and find the minimum value for each tuple (`hid`, `eid`). Importantly, the size (`len()`) of these rows will not be the same as the size of the base cohort since some tuples (`hid`, `eid`) will not have any values in the GROUP BY statement. Therefore we have to left join back onto the base cohort at some point (below).

In [ ]:
old_scr_vals = old_scr_vals.sort_values(['pdc_hid', 'pdc_eid', 'res_before_admit'])

# We want lab results from 12 hours before admission to 180 days before admission
lab_result_time_min = 0.5 * 24 * 60 * 60
lab_result_time_max = 180 * 24 * 60 * 60

old_scr_single_full = old_scr_vals[(old_scr_vals['res_before_admit'] >= lab_result_time_min) & (old_scr_vals['res_before_admit'] < lab_result_time_max)]

first_col_rename_dict = {
    'age_res_time': 'first_age_res_time',
    'val_numeric': 'first_scr',
    'res_before_admit': 'first_res_before_admit'
}
old_scr_single_first = old_scr_single_full[[
    'pdc_hid', 
    'pdc_eid', 
    'sex',
    'age_res_time',
    'val_numeric',
    'res_before_admit'
]].groupby(['pdc_hid', 'pdc_eid', 'sex']).first().reset_index()

for original_col in first_col_rename_dict.keys():
    old_scr_single_first[first_col_rename_dict[original_col]] = old_scr_single_first[original_col].copy()
    old_scr_single_first.drop(original_col, axis=1, inplace=True)
    
old_scr_single_min = old_scr_single_full[[
    'pdc_hid', 
    'pdc_eid', 
    'sex',
    'res_before_admit'
]].groupby(['pdc_hid', 'pdc_eid', 'sex']).min().reset_index()

old_scr_single_min['min_res_before_admit'] = old_scr_single_min['res_before_admit'].copy()
old_scr_single_min.drop('res_before_admit', axis=1, inplace=True)

old_scr_single = old_scr_single_first.merge(
    old_scr_single_min,
    on=['pdc_hid', 'pdc_eid', 'sex']
)

In [ ]:
type(old_scr_single)

In [ ]:
if debug_count: print('Number of rows:', len(old_scr_single))
if debug_show: old_scr_single.head()

In [ ]:
len(old_scr_vals[(old_scr_vals['res_before_admit'] >= lab_result_time_min) & (old_scr_vals['res_before_admit'] < lab_result_time_max)])

In [ ]:

fig = px.box(old_scr_vals[["site_id", "age_res_time"]], x="site_id", y="age_res_time", 
    labels={
        "site_id": "Site ID",
        "val_numeric": "Serum Creatinine Value"
    },
    log_y=True)
fig.update_xaxes(type='category')
fig.show()

#### Schwartz Values

Below we import the values from Schwartz 1976, which are used to minimum-check the baseline SCr values.

In [ ]:
columns = ['age', 'sex', 'mean', 'sd']
data = [
    (1, 'Female', 0.35, 0.05), (1, 'Male', 0.41, 0.10),
    (2, 'Female', 0.45, 0.07), (2, 'Male', 0.43, 0.12),
    (3, 'Female', 0.42, 0.08), (3, 'Male', 0.46, 0.11),
    (4, 'Female', 0.47, 0.12), (4, 'Male', 0.45, 0.11),
    (5, 'Female', 0.46, 0.11), (5, 'Male', 0.50, 0.11),
    (6, 'Female', 0.48, 0.11), (6, 'Male', 0.52, 0.12),
    (7, 'Female', 0.53, 0.12), (7, 'Male', 0.54, 0.14),
    (8, 'Female', 0.53, 0.11), (8, 'Male', 0.57, 0.16),
    (9, 'Female', 0.55, 0.11), (9, 'Male', 0.59, 0.16),
    (10, 'Female', 0.55, 0.13), (10, 'Male', 0.61, 0.22),
    (11, 'Female', 0.60, 0.13), (11, 'Male', 0.62, 0.14),
    (12, 'Female', 0.59, 0.13), (12, 'Male', 0.65, 0.16),
    (13, 'Female', 0.62, 0.14), (13, 'Male', 0.68, 0.21),
    (14, 'Female', 0.65, 0.13), (14, 'Male', 0.72, 0.24),
    (15, 'Female', 0.67, 0.22), (15, 'Male', 0.76, 0.22),
    (16, 'Female', 0.65, 0.15), (16, 'Male', 0.74, 0.23),
    (17, 'Female', 0.70, 0.20), (17, 'Male', 0.80, 0.18),
    (18, 'Female', 0.72, 0.19), (18, 'Male', 0.91, 0.17)
]

schwartz_df = spark.createDataFrame(data).toDF(*columns)
schwartz_df = schwartz_df.withColumn('min_value', schwartz_df['mean'] - 2. * schwartz_df['sd'])

if debug_show: schwartz_df.show()

Now we find the integer age (in years) from the `old_scr_single` data frame, and marge on age and gender to get the minimum value. If the actual value is less than the minimum, use the minimum from Schwarz 1976. This is per Sanchez-Pinto 2016 paper.

In [ ]:
def get_integer_age(age_val):
    return int(min(max(numpy.round(age_val), 1), 18))

baseline_scr_single = old_scr_single.copy()

baseline_scr_single['age'] = [get_integer_age(age) for age in baseline_scr_single['first_age_res_time']]

baseline_scr_single = baseline_scr_single.merge(
    schwartz_df.selectExpr("age", "sex", "min_value AS schwartz_val").toPandas(),
    on=['age', 'sex'],
    how="inner"
)

baseline_scr_single['baseline_scr'] = numpy.where(
    baseline_scr_single['first_scr'] < baseline_scr_single['schwartz_val'],
    baseline_scr_single['schwartz_val'], # if true
    baseline_scr_single['first_scr']     # else
)
baseline_scr_single['baseline_src'] = numpy.where(
    baseline_scr_single['first_scr'] < baseline_scr_single['schwartz_val'],
    'schwartz', # if true
    'prior'     # else
)

baseline_scr_single = baseline_scr_single[[
    'pdc_hid',
    'pdc_eid',
    'baseline_scr',
    'baseline_src'
]]

In [ ]:
if debug_count: print('Number of rows in baseline_scr_single:',len(baseline_scr_single))
if debug_show: baseline_scr_single.head()

#### Height Dependent & Independent

Lastly we need to calculate the height-dependent (when available) and height-independent values of SCr to be used as baselines, if the baseline values above do not exist (as we know they don't for a large portion of the cohort).

To do this we use the Hessey 2017 paper, which includes equations for both height-dependent (from Schwartz/Furth 2009) and height-independent (from Hoste 2014). Both of these require an eGFR, which we use from a lookup table (if age <= 2) or 120 ml/min/1.73m2 if above the age of 2. The lookup table is from the Hessey 2017 article's supplementary data.

In [ ]:
columns = ['age_min', 'age_max', 'egfr']

data = [
    (0., 0.1, 42),
    (0.1, 0.3, 53),
    (0.3, 0.66, 71),
    (0.66, 1.00, 84),
    (1., 1.5, 91),
    (1.5, 2., 97),
    (2., 100., 120)
]

# egfr_df = spark.createDataFrame(data).toDF(*columns)
egfr_df = pd.DataFrame(
    data=data,
    columns=columns
)

if debug_show: egfr_df.head()

Define the functions to return the height-dependent and height-independent values:

In [ ]:
def height_dependent(egfr, height_cm):
    """
    Height-dependent metohd, from Schwartz/Furth 2009,
    requires egfr and height (in cm).
    """

    return (0.41 * height_cm) / egfr

def height_independent_male(egfr, age_yrs):
    """
    Height-independent method (male), from Hoste 2014,
    requires egfr and age (in years)
    """

    Q = 0.21 \
        + 0.057 * age_yrs \
        - 0.0075 * (age_yrs*age_yrs) \
        + 0.00064 * (age_yrs*age_yrs*age_yrs) \
        - 0.000016 * (age_yrs*age_yrs*age_yrs*age_yrs)
    
    return (107.3 * Q) / egfr

def height_independent_female (egfr, age_yrs):
    """
    Height independent method (Female), from Hoste 2014, 
    requires egfr & age (in years) 
    """

    Q = 0.23 \
        + 0.034 * age_yrs \
        - 0.0018 * (age_yrs*age_yrs) \
        + 0.00017 * (age_yrs*age_yrs*age_yrs) \
        - 0.0000051 * (age_yrs*age_yrs*age_yrs*age_yrs)

    return (107.3 * Q) / egfr

Now we gather the required elements - we need a data table with the `base_cohort` primary key tuple (`pdc_hid`, `pdc_eid`) as well as `age` and `sex`. From this we left join to `height` (if not `None`), and then we join to `egfr` to find baseline egfr for each of these. Lastly, we apply both height-dependent and height-independent functions to these values.

In [ ]:
func_baseline = base_cohort[[
    'pdc_hid',
    'pdc_eid',
    'site_id',
    'ageatadmission',
    'sex'
]]
func_baseline['age_yrs'] = func_baseline['ageatadmission'] / 365.25
func_baseline = func_baseline[func_baseline['age_yrs'] <= 18]

if debug_count: print('Number of rows in base_cohort: %d' % len(func_baseline))

func_baseline = func_baseline.merge(
    height_join_lim_mod.toPandas(),
    on=['pdc_hid', 'pdc_eid', 'site_id'],
    how='left'
)

if debug_count: print('Number of rows in base_cohort joined to heights: %d' % len(func_baseline))

func_baseline['join_col'] = ['join_all' for i in range(len(func_baseline))]
egfr_df['join_col'] = ['join_all' for i in range(len(egfr_df))]

func_baseline = func_baseline.merge(
    egfr_df,
    on='join_col',
    how='outer'
)[[
    'pdc_hid',
    'pdc_eid',
    'site_id',
    'sex',
    'age_yrs',
    'age_min',
    'age_max',
    'val_height',
    'egfr'
]]

func_baseline = func_baseline[(func_baseline['age_yrs'] >= func_baseline['age_min']) & (func_baseline['age_yrs'] < func_baseline['age_max'])]
# func_baseline = func_baseline[(func_baseline['age_yrs'] >= func_baseline['age_min'])]
# func_baseline = func_baseline[(func_baseline['age_yrs'] < func_baseline['age_max'])]

func_baseline['val_height'] = func_baseline['val_height'].astype(float)

if debug_count: print('Number of rows joined to egfr: %d' % len(func_baseline))

if debug_show: func_baseline.head()

In [ ]:
func_baseline.head().T

Verify that we don't have any NULL egfr (and see how many NULL heights we have):

In [ ]:
if debug_count: 
    print('# of rows total:', len(func_baseline))

    # print('# of rows with NULL height:',func_baseline.filter(isnull(col('val_height'))).count())
    print('# of rows with NULL height:',len(func_baseline[func_baseline['val_height'].isna()]))

    # print('# of rows with NULL egfr:',func_baseline.filter(isnull(col('egfr'))).count())
    print('# of rows with NULL egfr:',len(func_baseline[func_baseline['egfr'].isna()]))

Apply the height-dependent and height-independent functions across this cohort:

In [ ]:
calc_bscr_with_heights = func_baseline[~func_baseline['val_height'].isna()].copy()
calc_bscr_without_heights_male = func_baseline[(func_baseline['val_height'].isna()) & (func_baseline['sex']=='Male')].copy()
calc_bscr_without_heights_female = func_baseline[(func_baseline['val_height'].isna()) & (func_baseline['sex']=='Female')].copy()

calc_bscr_with_heights['bscr'] = height_dependent(calc_bscr_with_heights['egfr'], calc_bscr_with_heights['val_height'])
calc_bscr_with_heights['bscr_source'] = ['height_dependent' for i in range(len(calc_bscr_with_heights))]

calc_bscr_without_heights_male['bscr'] = height_independent_male(calc_bscr_without_heights_male['egfr'], calc_bscr_without_heights_male['age_yrs'])
calc_bscr_without_heights_male['bscr_source'] = ['height_independent' for i in range(len(calc_bscr_without_heights_male))]

calc_bscr_without_heights_female['bscr'] = height_independent_female(calc_bscr_without_heights_female['egfr'], calc_bscr_without_heights_female['age_yrs'])
calc_bscr_without_heights_female['bscr_source'] = ['height_independent' for i in range(len(calc_bscr_without_heights_female))]

joined_bscr = pd.concat([calc_bscr_with_heights, calc_bscr_without_heights_male, calc_bscr_without_heights_female])

In [ ]:
func_baseline['sex'].unique()

In [ ]:
len(func_baseline[~func_baseline['sex'].isin(['Male', 'Female'])])

In [ ]:
len(calc_bscr_with_heights) + len(calc_bscr_without_heights_male) + len(calc_bscr_without_heights_female)

In [ ]:
if debug_count: print('Number of rows in joined_bscr table:', len(joined_bscr))

# Summarize by type
if debug_show:
    mean_by_source = joined_bscr[['bscr_source', 'bscr']].groupby('bscr_source').mean().reset_index()
    mean_by_source.rename(columns={'bscr_mean': 'bscr'}, inplace=True)
    count_by_source = joined_bscr[['bscr_source', 'bscr']].groupby('bscr_source').count().reset_index()
    count_by_source.rename(columns={'count': 'bscr'}, inplace=True)
    mean_by_source.merge(
        count_by_source,
        on='bscr_source',
        how='inner'
    )
    
if debug_show:
    joined_bscr.head(10)

## Export data to use in next notebook

In [ ]:
joined_bscr.to_csv(os.environ['AKI_DATA_PDC'] + 'joined_bscr.csv', index=False)
scr_join_left_icu.to_csv(os.environ['AKI_DATA_PDC'] + 'scr_join_left_icu.csv', index=False)